# Stage 2: Wound Type Classification Training

This notebook trains the **Stage 2** model, which classifies wounds into specific types (e.g., Diabetic Foot, Burn, Cut, etc.).

**Model:** EfficientNet-B0 (via `timm`)
**Classes:** Abrasion, Bruise, Burn, Cut, Diabetic Foot, Laceration, Surgical
**Input:** Filtered dataset (excluding 'healthy' and background)


In [1]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import timm
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Add project root to path for imports if needed
PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

f:\Housepital-AI\Housepital-AI\venv_gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
CONFIG = {
    "seed": 42,
    "img_size": 224,
    "batch_size": 8,
    "num_workers": 0,
    "epochs": 15,
    "lr": 1e-3,
    "model_name": "tf_efficientnet_b0",
    "model_dir": PROJECT_ROOT / "models" / "stage2_type",
    "data_train_csv": PROJECT_ROOT / "data" / "loaders" / "wound_type_train.csv",
    "data_val_csv": PROJECT_ROOT / "data" / "loaders" / "wound_type_val.csv",
    "root_dir": PROJECT_ROOT
}

# Ensure model directory exists
os.makedirs(CONFIG["model_dir"], exist_ok=True)

print(f"Model Directory: {CONFIG['model_dir']}")

Model Directory: F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\models\stage2_type


## Setup & Utils

In [3]:
# Class Mapping (Alphabetical)
CLASS_NAMES = ['abrasion', 'bruise', 'burn', 'cut', 'diabetic_foot', 'laceration', 'surgical']
CLASS_TO_IDX = {name: i for i, name in enumerate(CLASS_NAMES)}
IDX_TO_CLASS = {i: name for i, name in enumerate(CLASS_NAMES)}

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(CONFIG['seed'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using Device: {device}")

Using Device: cuda


## Dataset Class

In [ ]:
class WoundTypeDataset(Dataset):
    def __init__(self, df, root_dir=None, transform=None):
        self.annotations = df.copy()
        
        # Filter out 'healthy' and any unknown classes
        self.annotations = self.annotations[self.annotations['class'].isin(CLASS_NAMES)].reset_index(drop=True)
        
        self.root_dir = Path(root_dir) if root_dir else Path(".")
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        row = self.annotations.iloc[index]
        rel_path = row['path']
        rel_path = str(rel_path).replace('\\', os.sep).replace('/', os.sep)
        
        # Handle '..' prefix
        if rel_path.startswith(".."):
             img_path = self.root_dir / rel_path[3:]
        else:
             img_path = self.root_dir / rel_path
        
        # Fallback
        if not img_path.exists():
            # Try raw concatenation just in case
            img_path = self.root_dir / rel_path

        # Debug print for first few items
        # if index < 5:
            #  print(f"DEBUG: Loading {img_path}")
             
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # print(f"Warning: Could not open {img_path}: {e}, using black image.") # Reduce noise
            image = Image.new('RGB', (224, 224), color='black')
            
        label_str = row['class'] 
        label = CLASS_TO_IDX[label_str]
        
        if self.transform:
            image = np.array(image)
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image, torch.tensor(label, dtype=torch.long)

## Training & Validation Functions

In [5]:
def train_one_epoch(model, loader, criterion, optimizer, device, scaler=None):
    model.train()
    running_loss = 0.0
    preds_all = []
    targets_all = []
    
    pbar = tqdm(loader, desc="Training", leave=False)
    
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast(enabled=(scaler is not None)):
            outputs = model(images)
            loss = criterion(outputs, labels)
            
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
            
        running_loss += loss.item() * images.size(0)
        
        preds = torch.argmax(outputs, dim=1)
        preds_all.extend(preds.detach().cpu().numpy())
        targets_all.extend(labels.detach().cpu().numpy())
        
        pbar.set_postfix(loss=loss.item())
        
    epoch_loss = running_loss / len(loader.dataset)
    acc = accuracy_score(targets_all, preds_all)
    
    return epoch_loss, acc

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    preds_all = []
    targets_all = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim=1)
            preds_all.extend(preds.cpu().numpy())
            targets_all.extend(labels.cpu().numpy())
            
    epoch_loss = running_loss / len(loader.dataset)
    acc = accuracy_score(targets_all, preds_all)
    f1 = f1_score(targets_all, preds_all, average='macro')
    
    return epoch_loss, acc, f1

## Main Execution

In [6]:
# Transforms
train_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.2), # Cutout
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Load Dataframes
train_full = pd.read_csv(CONFIG['data_train_csv'])
val_full = pd.read_csv(CONFIG['data_val_csv'])

# Create Datasets
train_ds = WoundTypeDataset(train_full, root_dir=CONFIG['root_dir'], transform=train_transforms)
val_ds = WoundTypeDataset(val_full, root_dir=CONFIG['root_dir'], transform=val_transforms)

print(f"Train Samples (Filtered): {len(train_ds)}")
print(f"Val Samples (Filtered): {len(val_ds)}")

train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=CONFIG['num_workers'])
val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'])

# Model
print(f"Creating Model: {CONFIG['model_name']} with {len(CLASS_NAMES)} classes")
model = timm.create_model(CONFIG['model_name'], pretrained=True, num_classes=len(CLASS_NAMES)).to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])
scaler = torch.cuda.amp.GradScaler()

Train Samples (Filtered): 16417
Val Samples (Filtered): 4105
Creating Model: tf_efficientnet_b0 with 7 classes


f:\Housepital-AI\Housepital-AI\venv_gpu\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\Aly\AppData\Local\Temp\ipykernel_12936\4041318949.py:7: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.2), # Cutout
C:\Users\Aly\AppData\Local\Temp\ipykernel_12936\4041318949.py:39: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [7]:
# Training Loop
best_acc = 0.0
save_path = CONFIG['model_dir'] / "best_model.pth"

print(f"Starting training... Results will be saved to {save_path}")

for epoch in range(CONFIG['epochs']):
    print(f"\nEpoch {epoch+1}/{CONFIG['epochs']}")
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device, scaler)
    val_loss, val_acc, val_f1 = validate(model, val_loader, criterion, device)
    
    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")
    
    if val_acc > best_acc:
         print(f"🔥 Accuracy Improved ({best_acc:.4f} -> {val_acc:.4f}). Saving Model...")
         best_acc = val_acc
         torch.save(model.state_dict(), save_path)
         
print("Training Complete!")

Starting training... Results will be saved to F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\models\stage2_type\best_model.pth

Epoch 1/15


Training:   0%|          | 0/2053 [00:00<?, ?it/s]C:\Users\Aly\AppData\Local\Temp\ipykernel_12936\1029633126.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
Training:   9%|▉         | 192/2053 [00:21<03:57,  7.85it/s, loss=0.0363] 

DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\diabetic_foot\diabetic_foot_00180.jpg


Training:  24%|██▍       | 493/2053 [01:06<03:39,  7.12it/s, loss=0.466]   

DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\cut\cut_06347.jpg


...

Training:  80%|███████▉  | 1642/2053 [02:36<00:38, 10.67it/s, loss=0.00772] 

DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\cut\cut_06347.jpg


Validating:   1%|          | 3/514 [00:00<00:19, 26.82it/s]                 

DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\cut\cut_00475.jpg
DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\diabetic_foot\diabetic_foot_03561.jpg
DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\cut\cut_07297.jpg
DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\cut\cut_00276.jpg
DEBUG: Loading F:\Housepital-AI\Housepital-AI\AI_Pipeline_V2\data\raw\type_classification\cut\cut_05237.jpg


Train Loss: 0.0783 | Acc: 0.9762
Val   Loss: 0.0700 | Acc: 0.9820 | F1: 0.9179
🔥 Accuracy Improved (0.9803 -> 0.9820). Saving Model...
Training Complete!
